In [1]:
from Lib.data_loader import DataLoader
from Lib.utils import mkdirs
from Lib.resnet_model import Resnet3DBuilder
from Lib.HistoryGraph import Historygraph
import Lib.image as img
import os

In [2]:
from math import ceil

In [3]:
from keras.optimizers import SGD

In [4]:
from keras.callbacks import ModelCheckpoint

In [5]:
target_size =(64,96)
nb_frames= 16
skip = 1
nb_classes = 27
batch_size = 64
input_shape =(nb_frames,) + target_size + (3,)

In [6]:
workers = 8
use_multiprocessing = False
max_queue_size = 20

In [7]:
import pandas as pd
import numpy as np
# Path_videos
#Path_labels
#Path_train
#Path_validation
#Path_test
# Path_videos
#Path_labels
#Path_train
#Path_validation
#Path_test
class DataLoader():
    def __init__(self, path_vid, path_labels, path_train=None, path_val=None, path_test=None):
        self.path_vid=path_vid
        self.path_labels=path_labels
        self.path_train=path_train
        self.path_val=path_val
        self.path_test=path_test
        
        
        self.get_labels(path_labels)
            
        if self.path_train:
            self.train_df= self.load_video_labels(self.path_train)
        if self.path_val:
            self.val_df= self.load_video_labels(self.path_val)
        if self.path_test:
            self.test_df= self.load_video_labels(self.path_test,mode="input")
            
            
    def get_labels(self, path_labels):
        
        
        self.labels_df = pd.read_csv(path_labels, names=['Label'])
        #Extract data from dataFrames
        self.labels= [str(label[0]) for label in self.labels_df.values]
        #print(self.labels)
        self.n_labels= len(self.labels)
        #print(self.n_labels)
        #create Dictionaries to convert label to int and backward
        self.label_to_int= dict(zip(self.labels, range(self.n_labels))) 
       # print(self.label_to_int)
        self.init_to_labels= dict(enumerate(self.labels))
        #print(self.init_to_labels)
        #dictionary={0:swipingLeft}
        
    
        
    def load_video_labels(self, path_subset,mode="label"):
        if mode == "input":
            names= ["video_id"]
        elif mode == "label":
            names =["video_id", "label"]
        
        
        
        df=pd.read_csv(path_subset, sep=';', names=names)
        
        if mode == "label":
            df = df[df.label.isin(self.labels)]
        print(df)
            
            
        return df
        

In [8]:
data_root = r"B:\Dataset\labels"
csv_labels = r"B:\Dataset\labels\jester-v1-labels.csv"
csv_train = r"B:\Dataset\labels\jester-v1-train.csv"
csv_val = r"B:\Dataset\labels\jester-v1-validation.csv"
csv_test =r"B:\Dataset\labels\jester-v1-test.csv"
data_vid=r"B:\Dataset\labels\videos"
model_name= "resent_3d_model"
data_model=r"B:\Dataset\labels\model"

In [9]:
path_model=os.path.join(data_root, data_model, model_name)
path_vid=os.path.join(data_root,data_vid)
path_labels=os.path.join(data_root,csv_labels)
path_train=os.path.join(data_root,csv_train)
path_val=os.path.join(data_root, csv_val)
path_test=os.path.join(data_root, csv_test)


In [11]:
data = DataLoader(path_vid, path_labels, path_train, path_val)
mkdirs(path_model, 0o755)
mkdirs(os.path.join(path_model, "graphs"), 0o755)

        video_id                      label
0          34870           Drumming Fingers
1          56557  Sliding Two Fingers Right
2         129112   Sliding Two Fingers Down
3          63861     Pulling Two Fingers In
4         131717     Sliding Two Fingers Up
...          ...                        ...
118557     75507               Swiping Down
118558     48433   Sliding Two Fingers Left
118559    146421  Sliding Two Fingers Right
118560     49514                   Thumb Up
118561      4502     Sliding Two Fingers Up

[118562 rows x 2 columns]
       video_id                      label
0          9223                   Thumb Up
1        107090   Pushing Two Fingers Away
2         42920               Swiping Left
3        106485                 Thumb Down
4        142201      Rolling Hand Backward
...         ...                        ...
14782     97044           Drumming Fingers
14783    136208  Sliding Two Fingers Right
14784     12180      Rolling Hand Backward
14785    119381

In [12]:
gen=img.ImageDataGenerator()
gen_train= gen.flow_video_from_dataframe(data.train_df,path_vid,path_classes= path_labels,x_col='video_id',y_col="labels", target_size=target_size,batch_size=batch_size,nb_frames=nb_frames, skip=skip, has_ext=True)
gen_val= gen.flow_video_from_dataframe(data.val_df,path_vid,path_classes= path_labels,x_col='video_id',y_col="labels", target_size=target_size,batch_size=batch_size,nb_frames=nb_frames, skip=skip, has_ext=True)

Found 118562 video folders belonging to 27 classes.
Found 14787 video folders belonging to 27 classes.


In [13]:
resnet_model = Resnet3DBuilder.build_resnet_101(input_shape, nb_classes, drop_rate = 0.5)
optimizer = SGD(lr=0.01, momentum=0.9, decay=0.0001, nesterov=False)
resnet_model.compile(optimizer = optimizer, loss="categorical_crossentrophy", metrics=["accuracy"])
model_file= os.path.join(path_model, "resnetmodel.hdf5")

In [14]:
model_checkpointer = ModelCheckpoint(model_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [15]:
history_graph = Historygraph(model_path_name = os.path.join(path_model, "graphs"))

In [16]:
nb_sample_train = data.train_df["video_id"].size
nb_sample_val = data.val_df["video_id"].size

In [17]:
resnet_model.fit(
                 gen_train,
                 steps_per_epoch = ceil(nb_sample_train/batch_size),
                 epochs=100,
                 validation_data=gen_val,
                 validation_steps=30,
                 shuffle=True,
                 verbose=1,
                 workers= workers,
                 max_queue_size= max_queue_size,
                 use_multiprocessing= use_multiprocessing,
                 callbacks=[model_checkpointer,history_graph])
                                
                         

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'B:\\Dataset\\labels\\videos\\39876'